In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import pickle

In [2]:
pickle_model = None
filename = './Pickled_Models/mean temperature_4_gbtCV.pkl'
with open(filename, 'rb') as file:
    pickle_model = pickle.load(file)
est = pickle_model.best_estimator_

C:\Users\moyersjm\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\moyersjm\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DummyRegressor from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\moyersjm\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\moyersjm\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.0.2 when using version 0.24.2. This might lead to breaking 

In [3]:
df = pd.read_csv('./data/wunderground_scraped_data_daily_presentation_clean.csv')
df.head(3)

,Date,Time,Temperature,Dew Point,Humidity,Wind Direction,Wind Speed,Wind Gust,Pressure,Precipitation,Condition
0,2022-5-10-12:53 AM,12:53 AM,71,61,70,S,14,0,29.36,0.0,Fair
1,2022-5-10-1:53 AM,1:53 AM,70,61,73,S,15,21,29.37,0.0,Fair
2,2022-5-10-2:53 AM,2:53 AM,69,61,75,S,9,0,29.39,0.0,Fair


In [4]:
df_1 = df.loc[df.Date.str.contains('2022-5-18')]

In [5]:
df_4 = df.loc[df.Date.str.contains('2022-5-15')]

In [6]:
df_7 = df.loc[df.Date.str.contains('2022-5-12')]

In [7]:
th_altitude = 499
th_latitude = 39.4697

In [8]:
est.feature_names_in_

array(['day_of_year', 'altitude', 'latitude', 'temperature_0_12',
       'temperature_0_13', 'temperature_0_14', 'temperature_0_15',
       'temperature_0_16', 'temperature_0_17', 'temperature_0_18',
       'temperature_0_19', 'temperature_0_20', 'temperature_0_21',
       'temperature_0_22', 'temperature_0_23', 'temperature_0_0',
       'temperature_0_1', 'temperature_0_2', 'temperature_0_3',
       'temperature_0_4', 'temperature_0_5', 'temperature_0_6',
       'temperature_0_7', 'temperature_0_8', 'temperature_0_9',
       'temperature_0_10', 'temperature_0_11', 'pressure_0_12',
       'pressure_0_13', 'pressure_0_14', 'pressure_0_15', 'pressure_0_16',
       'pressure_0_17', 'pressure_0_18', 'pressure_0_19', 'pressure_0_20',
       'pressure_0_21', 'pressure_0_22', 'pressure_0_23', 'pressure_0_0',
       'pressure_0_1', 'pressure_0_2', 'pressure_0_3', 'pressure_0_4',
       'pressure_0_5', 'pressure_0_6', 'pressure_0_7', 'pressure_0_8',
       'pressure_0_9', 'pressure_0_10', 'pres

In [9]:
df_4 = df_4.drop(['Date', 'Condition'],axis=1)

In [10]:
df_4.Time = pd.to_datetime(df_4.Time)

In [11]:
df_tmp_4 = pd.DataFrame(columns=est.feature_names_in_)
df_tmp_4['day_of_year'] = [134]
df_tmp_4['altitude'] = [th_altitude]
df_tmp_4['latitude'] = [th_latitude]

In [12]:
df_4['Wind Direction'] = df_4['Wind Direction'].replace(\
    ['CALM','N','NNW','NW','WNW','W','WSW','SW','SSW','S','SSE','SE','ESE','E','ENE','NE','NNE','VAR'],
    [0, 0, 337.5, 315, 292.5, 270, 247.5, 225, 202.5, 180, 157.5, 135, 112.5, 90, 67.5, 45, 22.5, 0])

In [13]:
for k in range(24):
    col = f"temperature_0_{k}"
    df_tmp_4[col] = ((df_4.loc[df_4.Time.dt.hour == k].Temperature - 32) * (5/9) + 273.15).mean() # Convert to K
    col = f"pressure_0_{k}"
    df_tmp_4[col] = df_4.loc[df_4.Time.dt.hour == k].Pressure.mean()
    col = f"wind_speed_0_{k}"
    df_tmp_4[col] = df_4.loc[df_4.Time.dt.hour == k]['Wind Speed'].mean()
    col = f"humidity_0_{k}"
    df_tmp_4[col] = df_4.loc[df_4.Time.dt.hour == k].Humidity.mean()
    col = f"wind_direction_0_{k}"
    df_tmp_4[col] = df_4.loc[df_4.Time.dt.hour == k]['Wind Direction'].mean()
df_tmp_4.head()

,day_of_year,altitude,latitude,temperature_0_12,temperature_0_13,temperature_0_14,temperature_0_15,temperature_0_16,temperature_0_17,temperature_0_18,...,wind_direction_0_2,wind_direction_0_3,wind_direction_0_4,wind_direction_0_5,wind_direction_0_6,wind_direction_0_7,wind_direction_0_8,wind_direction_0_9,wind_direction_0_10,wind_direction_0_11
0,134,499,39.4697,299.816667,301.483333,302.594444,302.594444,302.594444,297.594444,293.372222,...,0.0,0.0,0.0,15.0,0.0,0.0,180.0,0.0,0.0,0.0


In [14]:
est.predict(df_tmp_4)

AttributeError: 'GradientBoostingRegressor' object has no attribute 'n_features_'

In [15]:
est.feature_names_in_.shape

(123,)